In [5]:
import json
import pandas as pd
import hdbscan
import numpy as np
from TFTClusterer import TFTClusterer
from save_matchdata import loaddb
from datetime import datetime, timedelta

In [6]:
db = loaddb(timestamp=(datetime.now() - timedelta(hours=24)).timestamp()*1000)
print(len(db))

11995


In [3]:
test=TFTClusterer(db)

In [4]:
test.cluster()

HDB Scan


In [4]:
test.unitshdb.to_csv("unitshdb.csv",index=False)
test.itemshdb.to_csv("itemshdb.csv",index=False)
test.traitshdb.to_csv("traitshdb.csv",index=False)
test.unitshdb[["comp_id","participants.placement","hdb","game_variation"]].drop_duplicates().to_csv("hdb.csv",index=False)

NameError: name 'test' is not defined

<module 'pbixrefresher' from 'C:\\Users\\JJYJa\\Anaconda3\\lib\\site-packages\\pbixrefresher\\__init__.py'>